3.9 多层感知机的从零开始实现
我们已经从上一节里了解了多层感知机的原理。下面，我们一起来动手实现一个多层感知机。首先导入实现所需的包或模块。


In [ ]:
import tensorflow as tf
import numpy as np
import sys

print(tf.__version__)

2.19.0


3.9.1 获取和读取数据
这里继续使用 Fashion-MNIST 数据集。我们将使用多层感知机对图像进行分类


In [10]:
# from tensorflow.keras.datasets import fashion_mnist
import os
import gzip
import numpy as np

data_dir = "./Data"


train_images_path = os.path.join(data_dir, "train-images-idx3-ubyte.gz")
train_labels_path = os.path.join(data_dir, "train-labels-idx1-ubyte.gz")
test_images_path = os.path.join(data_dir, "t10k-images-idx3-ubyte.gz")
test_labels_path = os.path.join(data_dir, "t10k-labels-idx1-ubyte.gz")


def load_images(filename):
    with gzip.open(filename, "rb") as f:
        data = np.frombuffer(f.read(), np.uint8, offset=16)  # 跳过前16字节头文件
    return data.reshape(-1, 28, 28)  # Fashion-MNIST 图像尺寸为 28x28


def load_labels(filename):
    with gzip.open(filename, "rb") as f:
        data = np.frombuffer(f.read(), np.uint8, offset=8)  # 跳过前8字节头文件
    return data


# 加载数据
x_train = load_images(train_images_path)
y_train = load_labels(train_labels_path)
x_test = load_images(test_images_path)
y_test = load_labels(test_labels_path)


batch_size = 256


x_train = tf.cast(x_train, tf.float32)


x_test = tf.cast(x_test, tf.float32)


x_train = x_train / 255.0


x_test = x_test / 255.0


train_iter = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)


test_iter = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

3.9.2 定义模型参数
我们在 3.6 节（softmax 回归的从零开始实现）里已经介绍了，Fashion-MNIST 数据集中图像形状为 28×28，类别数为 10。本节中我们依然使用长度为 28×28=784 的向量表示每一张图像。因此，输入个数为 784，输出个数为 10。实验中，我们设超参数隐藏单元个数为 256。


In [11]:
num_inputs, num_outputs, num_hiddens = 784, 10, 256

w1 = tf.Variable(tf.random.truncated_normal([num_inputs, num_hiddens], stddev=0.1))
b1 = tf.Variable(tf.random.truncated_normal([num_hiddens], stddev=0.1))
w2 = tf.Variable(tf.random.truncated_normal([num_hiddens, num_outputs], stddev=0.1))
b2 = tf.Variable(tf.random.truncated_normal([num_outputs], stddev=0.1))

3.9.3 定义激活函数
这里我们使用基础的 max 函数来实现 ReLU，而非直接调用 relu 函数。


In [12]:
def relu(x):
    return tf.math.maximum(x, 0)

In [13]:
def net(x, w1, b1, w2, b2):
    x = tf.reshape(x, shape=[-1, num_inputs])
    h = relu(tf.matmul(x, w1) + b1)
    y = tf.math.softmax(tf.matmul(h, w2) + b2)
    return y

3.9.5. 定义损失函数¶
为了得到更好的数值稳定性，我们直接使用 Tensorflow 提供的包括 softmax 运算和交叉熵损失计算的函数。


In [14]:
def loss(y_hat, y_true):
    return tf.losses.sparse_categorical_crossentropy(y_true, y_hat)

3.9.6. 训练模型


In [15]:
def acc(y_hat, y):
    return np.mean((tf.argmax(y_hat, axis=1) == y))

In [16]:
num_epochs, lr = 5, 0.5

In [17]:
for epoch in range(num_epochs):
    loss_all = 0
    for x, y in train_iter:
        with tf.GradientTape() as tape:
            y_hat = net(x, w1, b1, w2, b2)
            l = tf.reduce_mean(loss(y_hat, y))
            loss_all += l.numpy()
            grads = tape.gradient(l, [w1, b1, w2, b2])
            w1.assign_sub(grads[0])
            b1.assign_sub(grads[1])
            w2.assign_sub(grads[2])
            b2.assign_sub(grads[3])
    print(epoch, "loss:", l.numpy())
    total_correct, total_number = 0, 0

    for x, y in test_iter:
        with tf.GradientTape() as tape:
            y_hat = net(x, w1, b1, w2, b2)
            y = tf.cast(y, "int64")
            correct = acc(y_hat, y)
    print(epoch, "test_acc:", correct)

0 loss: 1.8563188
0 test_acc: 0.375
1 loss: 0.9739267
1 test_acc: 0.875
2 loss: 0.7761864
2 test_acc: 0.8125
3 loss: 0.7635223
3 test_acc: 0.9375
4 loss: 0.65596664
4 test_acc: 0.875
